In [ ]:
pip install BeautifulSoup

In [ ]:
pip install selenium

# 원래 코드

In [1]:
import os
from time import sleep
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


seoul_gu_list = ['마포구']
search_list = ['고기 맛집', '초밥', '카페']

# 시작 시간
start_time = time.time()
for index, store_name in enumerate(search_list):
    fileName = store_name + '_original.csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("매장명" + "|" + "주소" + "|" + "전화번호" + "|" + "\n")
    file.close()

    for index, gu_name in enumerate(seoul_gu_list):

        options = webdriver.ChromeOptions()
        options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
        options.add_argument('lang=ko_KR')
        chromedriver_path = "C:/Users/soiej/chromedriver_win32/chromedriver.exe"
        driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
        driver.get('https://map.kakao.com/')  # 주소 가져오기
        search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]') # 검색 창
        search_area.send_keys(gu_name + ' ' +store_name)  # 검색어 입력
        driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
        driver.implicitly_wait(3)
        time.sleep(1)
        more_page = driver.find_element(By.ID,"info.search.place.more")
        place_lists = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.place\.list > li')
        no_place = driver.find_element(By.ID,"info.noPlace")

        # 검색결과가 없을 때
        if 'HIDDEN' not in no_place.get_attribute('class'):
            continue

        # 더보기 버튼이 있을 때
        if 'HIDDEN' not in  more_page.get_attribute('class'):
            print("더보기 있음")
            more_page.send_keys(Keys.ENTER) # 더보기 버튼 선택
        # 더보기 버튼이 없을 때 (검색결과가 적을 때)
        else:
            file = open(fileName, 'a', encoding='utf-8')
            for p in place_lists: # WebElement
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                if len(place_name) == 0:
                    continue # 광고
                    
                place_naming = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                if gu_name not in place_address :
                    continue
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                # if len(place_tel) == 0 :
                #     continue

                file.write(place_naming + "|" + place_address + "|" + place_tel + "\n")
            continue

        time.sleep(1)

        Page = 1

        while True: # 다음 페이지가 있으면 loop
            file = open(fileName, 'a', encoding='utf-8')
            time.sleep(1)
            page_links = driver.find_elements(By.CSS_SELECTOR,"#info\.search\.page a")
            pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
            print(len(pages), "개의 페이지 있음")
            # pages를 하나씩 클릭하면서
            for i in range(1, len(pages) + 1):
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                try:
                    page = driver.find_element(By.XPATH,xPath)
                    page.send_keys(Keys.ENTER)
                except ElementNotInteractableException:
                    print('마지막 페이지')
                    break;
                sleep(3)
                place_lists = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.place\.list > li')
                for p in place_lists: # WebElement

                    store_html = p.get_attribute('innerHTML')
                    store_info = BeautifulSoup(store_html, "html.parser")

                    place_name = store_info.select('.head_item > .tit_name > .link_name')
                    if len(place_name) == 0:
                        continue # 광고
                    
            
                    place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                    place_address = store_info.select('.info_item > .addr > p')[0].text
                    if gu_name not in place_address :
                        continue
                    place_tel = store_info.select('.info_item > .contact > span')[0].text
                    # 전화번호가 없으면 선택 x
                    # if len(place_tel) == 0 :
                    #     continue

                    file.write(place_name + "|" + place_address + "|" + place_tel + "\n")
            next_btn = driver.find_element(By.ID,"info.search.page.next")
            has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
            if not has_next:
                print('다음페이지 x')
                driver.close()
                file.close()
                break # 다음 페이지 없으니까 종료
            else: # 다음 페이지 있으면
                Page += 1
                next_btn.send_keys(Keys.ENTER)
        print("종료")
        
end_time = time.time()

total_time = end_time - start_time
print(total_time)

C:\Users\soiej\AppData\Local\Temp/ipykernel_11552/3011597232.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


더보기 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
4 개의 페이지 있음
다음페이지 x
종료
더보기 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
4 개의 페이지 있음
다음페이지 x
종료
더보기 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
4 개의 페이지 있음
다음페이지 x
종료
368.4483392238617


# 병렬처리

In [3]:
import os
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import concurrent.futures

seoul_gu_list = ['마포구']
search_list = ['고기 맛집', '초밥', '카페']

def crawl_page(gu_name, store_name):
    
    fileName = store_name + '_thread.csv'
    file = open(fileName, 'a', encoding='utf-8')

    for index, gu_name in enumerate(seoul_gu_list):

        options = webdriver.ChromeOptions()
        options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
        options.add_argument('lang=ko_KR')
        chromedriver_path = "C:/Users/soiej/chromedriver_win32/chromedriver.exe"
        driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
        driver.get('https://map.kakao.com/')  # 주소 가져오기
        search_area = driver.find_element(By.XPATH,'//*[@id="search.keyword.query"]') # 검색 창
        search_area.send_keys(gu_name + ' ' +store_name)  # 검색어 입력
        driver.find_element(By.XPATH,'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
        driver.implicitly_wait(3)
        time.sleep(1)
        more_page = driver.find_element(By.ID,"info.search.place.more")
        place_lists = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.place\.list > li')
        no_place = driver.find_element(By.ID,"info.noPlace")

        # 검색결과가 없을 때
        if 'HIDDEN' not in no_place.get_attribute('class'):
            continue

        # 더보기 버튼이 있을 때
        if 'HIDDEN' not in  more_page.get_attribute('class'):
            print("더보기 있음")
            more_page.send_keys(Keys.ENTER) # 더보기 버튼 선택
        # 더보기 버튼이 없을 때 (검색결과가 적을 때)
        else:
            file = open(fileName, 'a', encoding='utf-8')
            for p in place_lists: # WebElement
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                if len(place_name) == 0:
                    continue # 광고

                place_naming = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                if gu_name not in place_address :
                    continue
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                # if len(place_tel) == 0 :
                #     continue
                file.write(place_naming + "|" + place_address + "|" + place_tel + "\n")
            continue

        time.sleep(1)

        Page = 1

        while True: # 다음 페이지가 있으면 loop
            file = open(fileName, 'a', encoding='utf-8')
            time.sleep(1)
            page_links = driver.find_elements(By.CSS_SELECTOR,"#info\.search\.page a")
            pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
            print(len(pages), "개의 페이지 있음")
            # pages를 하나씩 클릭하면서
            for i in range(1, len(pages) + 1):
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                try:
                    page = driver.find_element(By.XPATH,xPath)
                    page.send_keys(Keys.ENTER)
                except ElementNotInteractableException:
                    print('마지막 페이지')
                    break;
                sleep(3)
                place_lists = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.place\.list > li')
                for p in place_lists: # WebElement

                    store_html = p.get_attribute('innerHTML')
                    store_info = BeautifulSoup(store_html, "html.parser")

                    place_name = store_info.select('.head_item > .tit_name > .link_name')
                    if len(place_name) == 0:
                        continue # 광고


                    place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                    place_address = store_info.select('.info_item > .addr > p')[0].text
                    if gu_name not in place_address :
                        continue
                    place_tel = store_info.select('.info_item > .contact > span')[0].text
                    # 전화번호가 없으면 선택 x
                    # if len(place_tel) == 0 :
                    #     continue
                    file.write(place_name + "|" + place_address + "|" + place_tel + "\n")

            next_btn = driver.find_element(By.ID,"info.search.page.next")
            has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
            if not has_next:
                print('다음페이지 x')
                driver.close()
                file.close()
                break # 다음 페이지 없으니까 종료
            else: # 다음 페이지 있으면
                Page += 1
                next_btn.send_keys(Keys.ENTER)
        print("종료")

# 시작 시간
start_time = time.time()

for index, store_name in enumerate(search_list):
    fileName = store_name + '_thread.csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("매장명" + "|" + "주소" + "|" + "전화번호" + "|" + "\n")
    file.close()
    
# 병렬 처리를 위한 ThreadPoolExecutor 생성
executor = concurrent.futures.ThreadPoolExecutor()

for store_name in search_list:
    for gu_name in seoul_gu_list:
        # 크롤링 작업을 병렬로 실행
        executor.submit(crawl_page, gu_name, store_name)

# 모든 작업이 완료될 때까지 대기
executor.shutdown()

end_time = time.time()

total_time = end_time - start_time
print(total_time)

C:\Users\soiej\AppData\Local\Temp/ipykernel_11552/924790157.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


더보기 있음
더보기 있음
더보기 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
5 개의 페이지 있음
4 개의 페이지 있음
4 개의 페이지 있음
4 개의 페이지 있음
다음페이지 x
종료
다음페이지 x
종료
다음페이지 x
종료
123.52267074584961
